In [212]:
import numpy as np
import matplotlib.pyplot as plt
from FD_Vlasov_supp import *
import importlib
importlib.import_module('mpl_toolkits.mplot3d').__path__
from mpl_toolkits.mplot3d import Axes3D
import scipy.integrate as integrate
import numpy.linalg as la

% matplotlib inline
np.set_printoptions(precision = 2)

Solve Vlasov's equation in one space dimension and one velocity dimension:
\begin{equation}
\frac{\partial f}{\partial t} + v\frac{\partial f}{\partial x} + E(t,x)\frac{\partial f}{\partial v} = 0
\end{equation}
With $x \in (0,L)$ and $v \in (-\infty, \infty)$, and $\lim_{|v|\to \infty} f(t,x,v) = 0$

We also need boundary conditions for $x$... which I'm not too sure about.  I'll try homogenous neumann for $x$ (closed domain)

We'll use a cutoff velocity $L_v$ to define our computational domain for $v$.  The electric field can be solved through:
\begin{align}
-\frac{\partial^2 \Phi}{\partial x^2}(t,x) &= \frac{q}{\epsilon_0}\int_{\mathbb{R}}f(t,x,v)dv\\
E(t,x) &= -\frac{\partial \Phi}{\partial x}
\end{align}
Again, I'm not too sure about boundary conditions for the potential...I'll say the potential is zero at the walls but this is iffy

Initial Condition here

We'll discretize in space and velocity first.  Upwinding needed since this is an advective problem (backward difference in $x$ for $v > 0$ and forward difference in $x$ for $v < 0$

In [239]:
Lv = 20     # approximation to "infinity" for velocity space

k = 0.483
L = 2*np.pi/k  # length of spatial domain, which is periodic


nx = 5
nv = 5   # 

x = np.linspace(0,L,nx)
v = np.linspace(-Lv,Lv,nv)

X,V = np.meshgrid(x,v)  # represent as grid

def initial_u(x,v):
    z = np.exp(-(v**2)/2)
    z = z/(np.sqrt(2*np.pi)*L)
    return z

U = initial_u(X,V)
U[1,:] = 0.0
U[-1,:] = 0.0

In [245]:
def space_derivative(u,dx,nv):

    mid = int(np.floor(nv/2))
    upper_U = u[:mid,:]
    lower_U = u[mid:,:]
    dudx = np.zeros_like(u)
    
    dudx[:mid,:] = np.roll(upper_U,-1,axis = 1) - upper_U
    dudx[mid:,:] = lower_U - np.roll(lower_U,1,axis = 1)
    dudx = dudx/dx
    
    # homogeneous neumann
    dudx[:,0] = 0
    dudx[:,-1] = 0
    
    return dudx[1:-1,:] # don't need to update Dirichlet B.C's

def velocity_derivative(u,dv,E): #upwinding???
    # will need to look at sign of E
    E_pos = np.where(E >= 0)
    E_neg = np.where(E < 0)
    
    dudv = np.zeros_like(u)
    dudv[:,E_pos] = dudv[:,E_pos] - np.roll(u,1,axis = 0)[:,E_pos]
    dudv[:,E_neg] = np.roll(u,-1,axis = 0)[:,E_pos] - dudv[:,E_neg]
    
    dudv = dudv/dv
    dudv = dudv/(dv[:,:-1])
    
    return dudv[1:-1,:]

def Forward_Euler_step(U,dt,E,V,dudx,dudv):
    Unew = U
    dudx = space_derivative(U,dx,)
    Unew[1:-1,:] = U[1:-1,:] + dt*(-V*dudx - E*dudv)
    return Unew

def integrate_u(u):
    int_u = np.zeros(nx,)
    
    # assume u has no redundant column
    int_u[1:] = integrate.simps(u,v,axis = 0)
    int_u[0] = int_u[-1]
    
    return int_u

In [246]:
bb = np.linspace(-1,1,6)
np.where(bb > 0)

(array([3, 4, 5]),)

In [252]:
x.shape

(5,)

In [253]:
x

array([  0.  ,   3.25,   6.5 ,   9.76,  13.01])

In [244]:
print(dudx[1,0])
print(2*x[1])

0.796448891771
6.50433261613


In [172]:
2*(x[1] - x[0])**2

0.14096240826730755